# GCP Professional Data Engineer
### Leveraging Unstructured Data with Cloud Dataproc on Google Cloud Platform

## Module 1: Introduction to Cloud Dataproc
### Introducing Cloud Dataproc
### Why Unstructured Data
### Why Cloud Dataproc
### Creating a Dataproc Cluster
### Lab: Create a Dataproc Cluster
### Module 1 Review

## Module 2: Running Dataproc Jobs
### Running Jobs
### Lab: Work with structured and semi-structured Data
### Separation of Storage & Compute
### Submitting Jobs
### Spark RDDs, Transformations, and Actions
### Lab: Working with Spark Jobs
### Module 2 Review

## Module 3: Leveraging GCP
### Big Query Support
### Lab: Leverage GCP
### Customizing Clusters
### Lab: Cluster Automation using CLI Commands
### Module 3 Review

## Module 4: Analyzing Unstructured Data
### Infuse Your Business With Machine Learning
### Lab: Add Machine Learning
### Module 4 Review